In [1]:
local_path = './'

"""## Prepare fastai"""
from fastai import *
from fastai.text import *
from fastai.metrics import *
torch.cuda.set_device(1)

"""## Prepare Dataset"""
local_project_path = local_path + 'bp_deepred/'
if not os.path.exists(local_project_path):
    os.makedirs(local_project_path)
print('local_project_path:', local_project_path)

"""## Create Language Model"""
class dna_tokenizer(BaseTokenizer):
    def tokenizer(slef, t):
        return list(t)
tokenizer = Tokenizer(tok_func=dna_tokenizer, pre_rules=[], post_rules=[], special_cases=[])
processor = [TokenizeProcessor(tokenizer=tokenizer, include_bos= True, include_eos=True), NumericalizeProcessor(max_vocab=30000)]

local_project_path: ./bp_deepred/


In [13]:
# batch size
bs = 2048
data_lm = TextLMDataBunch.from_csv(local_project_path, 'combined.csv',
                                   text_cols ='seq', valid_pct= 0.1, tokenizer=tokenizer,
                                   include_bos= True, include_eos=True, bs=bs)
print('data_cls Training set size', len(data_lm.train_ds))
print('data_cls Validation set size', len(data_lm.valid_ds))

data_cls Training set size 129303
data_cls Validation set size 14368


In [3]:
data_lm.show_batch()

idx,text
0,V Y A Y V R S C P G Y T L D Y N A N G S G A G V T Q F L N N E T D F A G S D V P L N P S T G Q P D R S A E R C G S P A W D L P T V F G P I A
1,I N V L V F A S M L L V I S K A L F S H A C E G R R R K W V F P T L V G F S L D E G A K P L D G A S R M K L T G L S P Y W F D V S S G T A V
2,L G E C R Q M S S S S Q I G N D S F Q L Q V T E D V N S S I K T F F C P N D T Y N D M A T L F F N P Q E S A I L Q L F H Q D G T F S P V T L
3,D P K T S L Y E I P A V S S S S F F E E F G K L L R E A D E M D S I H D V T F Q V G N R L F P A H K Y I L A V H S D F F Q K L F L S D G N T
4,x b o s M G V P S F F R W L S R K Y P K I I S P V L E E Q P Q I V D G V I L P L D Y S A S N P N G E L D N L Y L D M N G I V H P C S H P


In [ ]:
data_lm.vocab.itos

In [7]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3, pretrained=False).to_fp16()

In [14]:
learn_lm.data = data_lm

In [ ]:
learn_lm.lr_find()
learn_lm.recorder.plot(skip_end = 15)

In [ ]:
learn_lm.fit_one_cycle(1, 1e-2/2.6**1, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time


In [20]:
learn_lm.unfreeze()

In [ ]:
lr=0.01
learn_lm.fit_one_cycle(1, slice(lr), moms=(0.8,0.7))

In [ ]:
learn_lm.fit_one_cycle(1, slice(1e-2), moms=(0.8,0.7))